# Visual Questions Answering simplest examples

First we need to install easy-vqa this is the data set we will be using in our experiments for mor details follow the link:https://github.com/vzhou842/easy-VQA

easy-VQA contains
4,000 train images and 38,575 train questions.

*   1,000 test images and 9,673 test questions.
*   13 total possible answers.
*   28,407 training questions that are yes/no.
*   7,136 testing questions that are yes/no.

All images are 64x64 color images




In [ ]:
pip install easy-vqa

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 3.2 MB 4.7 MB/s 


# Making Data set Ready
In this section we have written the code to get the data set ready for our analysis.


1.   Reading Questions and there actual answers
2.   Reading all the images and saving there paths and data into arrays
3.   Getting Ready the training and testing data








In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.utils import to_categorical
import json
import os
import numpy as np
from easy_vqa import get_train_questions, get_test_questions, get_train_image_paths, get_test_image_paths, get_answers

def setup():
  print('\n--- Reading questions...')

  # Use the easy-vqa package
  train_qs, train_answers, train_image_ids = get_train_questions()
  test_qs, test_answers, test_image_ids = get_test_questions()
  # print("here1",test_qs[0])
  print(f'Read {len(train_qs)} training questions and {len(test_qs)} testing questions.')


  print('\n--- Reading answers...')
  # Read answers from the easy-vqa package
  all_answers = get_answers()
  num_answers = len(all_answers)
  print(f'Found {num_answers} total answers:')
  print(all_answers)


  print('\n--- Reading/processing images...')
  def load_and_proccess_image(image_path):
    # Load image, then scale and shift pixel values to [-0.5, 0.5]
    im = img_to_array(load_img(image_path))
    return im / 255 - 0.5

  def read_images(paths):
    # paths is a dict mapping image ID to image path
    # Returns a dict mapping image ID to the processed image
    ims = {}
    for image_id, image_path in paths.items():
      ims[image_id] = load_and_proccess_image(image_path)
    return ims


  # Read images from the easy-vqa package
  test_paths = get_test_image_paths()
  train_ims = read_images(get_train_image_paths())
  test_ims = read_images(get_test_image_paths())
  im_shape = train_ims[0].shape
  print(f'Read {len(train_ims)} training images and {len(test_ims)} testing images.')
  print(f'Each image has shape {im_shape}.')


  print('\n--- Fitting question tokenizer...')
  tokenizer = Tokenizer()
  tokenizer.fit_on_texts(train_qs)

  # We add one because the Keras Tokenizer reserves index 0 and never uses it.
  vocab_size = len(tokenizer.word_index) + 1
  print(f'Vocab Size: {vocab_size}')
  print(tokenizer.word_index)


  print('\n--- Converting questions to bags of words...')
  train_X_seqs = tokenizer.texts_to_matrix(train_qs)
  test_X_seqs = tokenizer.texts_to_matrix(test_qs)
  print(f'Example question bag of words: {test_X_seqs[0]}')


  print('\n--- Creating model input images...')
  train_X_ims = np.array([train_ims[id] for id in train_image_ids])
  test_X_ims = np.array([test_ims[id] for id in test_image_ids])


  print('\n--- Creating model outputs...')
  train_answer_indices = [all_answers.index(a) for a in train_answers]
  test_answer_indices = [all_answers.index(a) for a in test_answers]
  train_Y = to_categorical(train_answer_indices)
  test_Y = to_categorical(test_answer_indices)
  print(f'Example model output: {train_Y[0]}')

  return (train_X_ims, train_X_seqs, train_Y, test_X_ims, test_X_seqs,
          test_Y, im_shape, vocab_size, num_answers,
          all_answers, test_qs, test_answer_indices,test_paths,test_qs)  # for the analyze script


# Creating Model:
Here we create the Model for our Visual Question Answering this is one of the simplest model as the data set we are using is very simple.

The first part show the CNN model for Images while in the second part we used a fully connected layers for the questions. In the third part we have combined the two models for the prediction of the questions.

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Multiply
from tensorflow.keras.optimizers import Adam

def build_model(im_shape, vocab_size, num_answers):
  # The CNN
  im_input = Input(shape=im_shape)
  x1 = Conv2D(8, 3, padding='same')(im_input)
  x1 = MaxPooling2D()(x1)
  x1 = Conv2D(16, 3, padding='same')(x1)
  x1 = MaxPooling2D()(x1)
  x1 = Flatten()(x1)
  x1 = Dense(32, activation='tanh')(x1)

  # The question network
  q_input = Input(shape=(vocab_size,))
  x2 = Dense(32, activation='tanh')(q_input)
  x2 = Dense(32, activation='tanh')(x2)

  # Merge -> output
  out = Multiply()([x1, x2])
  out = Dense(32, activation='tanh')(out)
  out = Dense(num_answers, activation='softmax')(out)

  model = Model(inputs=[im_input, q_input], outputs=out)
  model.compile(Adam(learning_rate=5e-4), loss='categorical_crossentropy', metrics=['accuracy'])

  return model


# Training:
Now training our model using the data prepared intially

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint
import argparse

# Prepare data
train_X_ims, train_X_seqs, train_Y, test_X_ims, test_X_seqs, test_Y, im_shape, vocab_size, num_answers, _, _, _, _,_ = setup()

print('\n--- Building model...')
model = build_model(im_shape, vocab_size, num_answers)
checkpoint = ModelCheckpoint('model.h5', save_best_only=True)

print('\n--- Training model...')
model.fit(
  [train_X_ims, train_X_seqs],
  train_Y,
  validation_data=([test_X_ims, test_X_seqs], test_Y),
  shuffle=True,
  epochs=8,
  callbacks=[checkpoint],
)



--- Reading questions...
Read 38575 training questions and 9673 testing questions.

--- Reading answers...
Found 13 total answers:
['circle', 'green', 'red', 'gray', 'yes', 'teal', 'black', 'rectangle', 'yellow', 'triangle', 'brown', 'blue', 'no']

--- Reading/processing images...
Read 4000 training images and 1000 testing images.
Each image has shape (64, 64, 3).

--- Fitting question tokenizer...
Vocab Size: 27
{'is': 1, 'shape': 2, 'the': 3, 'a': 4, 'image': 5, 'there': 6, 'not': 7, 'what': 8, 'present': 9, 'does': 10, 'contain': 11, 'in': 12, 'color': 13, 'no': 14, 'circle': 15, 'rectangle': 16, 'triangle': 17, 'brown': 18, 'yellow': 19, 'gray': 20, 'teal': 21, 'black': 22, 'red': 23, 'green': 24, 'blue': 25, 'of': 26}

--- Converting questions to bags of words...
Example question bag of words: [0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.]

--- Creating model input images...

--- Creating model outputs...
Example model output: [0. 0. 0. 0. 0. 

# Analysis of our Experiments:
The following code is for performing analysis against our model
where the first part represent the accuracy stats against each type of the answers. The scond part represents Errors. In last part represents the questions which were answerd wrongly.

In [ ]:
import argparse
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

train_X_ims, train_X_seqs, train_Y, test_X_ims, test_X_seqs, test_Y, im_shape, vocab_size, num_answers, all_answers, test_qs, test_answer_indices,test_paths,test_qs = setup()

print('\n--- Building model...')
model = build_model(im_shape, vocab_size, num_answers,)

model.load_weights('model.h5')
predictions = model.predict([test_X_ims, test_X_seqs])

# Stats for each answer
for idx in range(num_answers):
	pred_values = predictions[:, idx]
	answer = all_answers[idx]
	print(f'\nStatistics for answer {idx}, {answer}')
	min = np.amin(pred_values)
	max = np.amax(pred_values)
	mean = np.mean(pred_values)
	print(f'Min: {min}, Max: {max}, Mean: {mean}')

shapes = []
yesno = []
for i in range(num_answers):
  if (all_answers[i] == 'rectangle' or all_answers[i] == 'circle' or all_answers[i] == 'triangle'):
    shapes.append(i)
  elif all_answers[i] == 'yes' or all_answers[i] == 'no':
    yesno.append(i)

def return_class(answer):
  if answer in shapes:
    return 0
  if answer in yesno:
    return 1
  return 2
error_matrix = [[0 for _ in range(3)] for _ in range(3)]
total_errors = 0

color_error_matrix = [[0 for _ in range(num_answers)] for _ in range(num_answers)]
questions_wrong = {}

for idx in range(len(test_answer_indices)):
  # answer numbers for triangle, circle, rectangle
  answer = test_answer_indices[idx]
  pred = np.argmax(predictions[idx])
  if not answer == pred:
    total_errors += 1
    error_matrix[return_class(answer)][return_class(pred)] += 1
    color_error_matrix[answer][pred] += 1
    if (return_class(answer) == 1 and return_class(pred) == 1):
      if test_qs[idx] in questions_wrong:
        questions_wrong[test_qs[idx]] += 1
      else:
        questions_wrong[test_qs[idx]] = 1

print('\n\ntotal error: {}'.format(total_errors / len(test_answer_indices)))
print('Indexes are, in order, shapes, yes/no, colors')
print('Rows are class of answer, columns are class of prediction')
for i in range(3):
  print('{}\t{}\t{}\n'.format(error_matrix[i][0] / total_errors, error_matrix[i][1] / total_errors, error_matrix[i][2]/ total_errors))
print('-------------')
for i in range(num_answers):
  to_print = ''
  for j in range(num_answers):
    to_print += str(color_error_matrix[i][j]) + '\t'
  print(to_print)
print('-------------')
print(questions_wrong)


# print('examples')

# img = mpimg.imread(test_paths[1])
# imgplot = plt.imshow(img)
# plt.show()
# print('Question Example :',test_qs[2])
# print(predictions.shape)
# pred = np.argmax(predictions[1,2])
# print('predicted answer :',pred)



--- Reading questions...
Read 38575 training questions and 9673 testing questions.

--- Reading answers...
Found 13 total answers:
['circle', 'green', 'red', 'gray', 'yes', 'teal', 'black', 'rectangle', 'yellow', 'triangle', 'brown', 'blue', 'no']

--- Reading/processing images...
Read 4000 training images and 1000 testing images.
Each image has shape (64, 64, 3).

--- Fitting question tokenizer...
Vocab Size: 27
{'is': 1, 'shape': 2, 'the': 3, 'a': 4, 'image': 5, 'there': 6, 'not': 7, 'what': 8, 'present': 9, 'does': 10, 'contain': 11, 'in': 12, 'color': 13, 'no': 14, 'circle': 15, 'rectangle': 16, 'triangle': 17, 'brown': 18, 'yellow': 19, 'gray': 20, 'teal': 21, 'black': 22, 'red': 23, 'green': 24, 'blue': 25, 'of': 26}

--- Converting questions to bags of words...
Example question bag of words: [0. 1. 1. 1. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1.
 0. 0. 0.]

--- Creating model input images...

--- Creating model outputs...
Example model output: [0. 0. 0. 0. 0. 